In [19]:
import tensorflow.keras as keras
import json
import numpy as np
from sklearn.model_selection import train_test_split

DATA_PATH = "/content/drive/My Drive/Dataset/speech.json"
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
EPOCHS = 40
MODEL_SAVE_PATH = "/content/drive/My Drive/Models/speech_model.h5"
NUM_KEYWORDS =12

In [20]:
import tensorflow as tf
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
  tpu =None

tpu
try: # detect TPUs
  tpu = None
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # detect GPUs
  #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
  #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.73.205.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.73.205.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8


In [39]:

def load_dataset(data_path):
  with open(data_path,'r') as op:
    data = json.load(op)
    x = np.array(data["MFCCs"])
    y = np.array(data["labels"])
    return x,y

def get_data_splits(data_path, test_size =0.1):
  data, labels = load_dataset(data_path)
  X_train,x_test,Y_train,y_test = train_test_split(data, labels, test_size = test_size)
  x_train, x_validation,y_train,y_validation = train_test_split(X_train,Y_train, test_size = test_size) 
  #2d -> 3d
  x_train = x_train[...,np.newaxis]
  x_test = x_test[...,np.newaxis]
  x_validation = x_validation[...,np.newaxis]
  return x_train,x_test,x_validation,y_train,y_test,y_validation
x_train,x_test,x_validation,y_train,y_test,y_validation=get_data_splits(DATA_PATH)


In [41]:
def build_model(input_shape,learning_rate,error = "sparse_categorical_crossentropy"):
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(64,(3,3),activation ="relu", input_shape =input_shape,
                              kernel_regularizer=keras.regularizers.l2(0.001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.MaxPool2D((2,2),strides = (2,2), padding = "same"))
  
  model.add(keras.layers.Conv2D(32,(3,3),activation ="relu",
                              kernel_regularizer=keras.regularizers.l2(0.001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.MaxPool2D((3,3),strides = (2,2), padding = "same"))

  model.add(keras.layers.Conv2D(32,(2,2),activation ="relu", 
                              kernel_regularizer=keras.regularizers.l2(0.001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.MaxPool2D((2,2),strides = (2,2), padding = "same"))
  
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64,activation = "relu"))
  model.add(keras.layers.Dropout(0.3))

  model.add(keras.layers.Dense(NUM_KEYWORDS, activation = "softmax"))


  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(optimizer = optimizer, loss = error, metrics=["accuracy"])

  model.summary()
  return model 
input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3]) 
model = build_model(input_shape,LEARNING_RATE)


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 42, 11, 64)        640       
_________________________________________________________________
batch_normalization_9 (Batch (None, 42, 11, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 21, 6, 64)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 19, 4, 32)         18464     
_________________________________________________________________
batch_normalization_10 (Batc (None, 19, 4, 32)         128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 10, 2, 32)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 9, 1, 32)         

In [44]:
import json
with open(DATA_PATH,'r') as op:
    data = json.load(op)
    print(data["mappings"])

['audio', 'zero', 'six', 'seven', 'two', 'one', 'three', 'four', 'left', 'stop', 'yes', 'five', 'eight']


In [42]:
def main():
  x_train,x_test,x_validation,y_train,y_test,y_validation=get_data_splits(DATA_PATH)
  
  input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3]) #(number of segments = total length of segment/hop length,
                                                                     # number of MFCCs , Diamentional image (1) )
  model = build_model(input_shape,LEARNING_RATE)

  model.fit(x_train,y_train, epochs =EPOCHS, batch_size = BATCH_SIZE,
            validation_data = (x_validation,y_validation))

  test_error, test_accuracy = model.evaluate(x_test,y_test)
  print(f"test error ; {test_error},test accuracy : {test_accuracy}")
  model.save(MODEL_SAVE_PATH)

if __name__=='__main__':
  main()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 42, 11, 64)        640       
_________________________________________________________________
batch_normalization_12 (Batc (None, 42, 11, 64)        256       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 21, 6, 64)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 19, 4, 32)         18464     
_________________________________________________________________
batch_normalization_13 (Batc (None, 19, 4, 32)         128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 10, 2, 32)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 9, 1, 32)         

654/656 [============================>.] - ETA: 0s - loss: 2.3639 - accuracy: 0.2564WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0007s vs `on_test_batch_end` time: 0.0040s). Check your callbacks.


656/656 [==============================] - 14s 21ms/step - loss: 2.3632 - accuracy: 0.2566 - val_loss: 1.6206 - val_accuracy: 0.5296
Epoch 2/40
656/656 [==============================] - 12s 19ms/step - loss: 1.5274 - accuracy: 0.5256 - val_loss: 1.0683 - val_accuracy: 0.7172
Epoch 3/40
656/656 [==============================] - 14s 21ms/step - loss: 1.1361 - accuracy: 0.6603 - val_loss: 0.7835 - val_accuracy: 0.8021
Epoch 4/40
656/656 [==============================] - 14s 21ms/step - loss: 0.8935 - accuracy: 0.7403 - val_loss: 0.6274 - val_accuracy: 0.8451
Epoch 5/40
656/656 [==============================] - 14s 21ms/step - loss: 0.7518 - accuracy: 0.7853 - val_loss: 0.5309 - val_accuracy: 0.8644
Epoch 6/40
656/656 [==============================] - 13s 19ms/step - loss: 0.6521 - accuracy: 0.8197 - val_loss: 0.4591 - val_accuracy: 0.8858
Epoch 7/40
656/656 [==============================] - 13s 20ms/step - loss: 0.5791 - accuracy: 0.8436 - val_loss: 0.4109 - val_accuracy: 0.8987
Epo

81/81 [==============================] - 1s 11ms/step - loss: 0.2519 - accuracy: 0.9386
test error ; 0.25194352865219116,test accuracy : 0.9385863542556763
